In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd


In [2]:
from app.flipside.models import Flipside
from app.oracles.models import Oracle
from app.constants import (
    oracle_feeds_lookup, oracle_feeds, 
    phase_windows, timestamp_windows,
    filenames_filtered
    )
from app.aave.models import AaveBalancePlotter
from app.combo.models import Combo
from app.swaps.models import DexSwaps
from app.visuals.plotly_helper import combined_action_layout, combined_action_layout_time

In [3]:
def fig_helper(fig, a=True, b=True, c=True, d=False):
    if a:
        fig.add_vrect(x0=timestamp_windows['michael_ramp_up'], x1=timestamp_windows['michael_ramp_stop'], 
                annotation_text='M: Setup', annotation_position="top left",
                fillcolor='blue', opacity=0.25, line_width=0)
    if b:
        fig.add_vrect(x0=timestamp_windows['avi_initial_deposit'], x1=timestamp_windows['avi_attack_start'], 
                annotation_text='A: Setup', annotation_position="top left",
                fillcolor='orange', opacity=0.25, line_width=0)
    if c:
        fig.add_vrect(x0=timestamp_windows['avi_attack_start'], x1=timestamp_windows['avi_attack_end'], 
                    annotation_text='Attack', annotation_position="top left",
                    fillcolor='red', opacity=0.25, line_width=0)
    if d:
        fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_last_liquidation'], 
                    annotation_text='A: Liq', annotation_position="top left",
                    fillcolor='purple', opacity=0.25, line_width=0)
    return fig





In [ ]:
fs = Flipside(filenames_filtered)

In [5]:
oracles = Oracle(fs, oracle_feeds_lookup)

In [6]:
crv_price_feed = pd.DataFrame(oracles.records['CRV'])


In [7]:
swaps = DexSwaps(fs)

volume_hourly = pd.DataFrame(swaps.hourly)
volume_daily = pd.DataFrame(swaps.daily)


In [8]:
aave = AaveBalancePlotter(fs)
aave.iterate_through_blocks()

In [9]:
target_address = '0x57e04786e231af3343562c062e0d058f25dace9e'
window_start = 15900000 - 420
window_end = 16027236 + 420

title = "Avi"

In [10]:
df_balance_tracker = aave.balance_tracker[aave.balance_tracker["ACTOR_ADDRESS"].isin([target_address])]

In [ ]:
combo = Combo(aave, oracles, target_address)
combo.process_value(window_start, window_end)
combo.process_sums()


In [12]:
df_combo_crv = pd.DataFrame(combo.records['CRV'])
df_combo_usdc = pd.DataFrame(combo.records['USDC'])

In [ ]:
liq = aave.liquidations
liq_filtered = liq.loc[
            (liq["BLOCK_NUMBER"] > 16025860 - 50) &
            (liq["BLOCK_NUMBER"] < 16027236 + 50)
            ]
liq_filtered = liq_filtered.sort_values("BLOCK_NUMBER", axis = 0, ascending = True)

liq_filtered

In [14]:
df_crv_filtered = df_combo_crv.loc[
            (df_combo_crv["block_number"] > 16025860 - 50) &
            (df_combo_crv["block_number"] < 16025896 + 50)
            ]

In [ ]:
df_usdc_filtered = df_combo_usdc.loc[
            (df_combo_usdc["block_number"] > 16027007 - 50) &
            (df_combo_usdc["block_number"] < 16027236 + 50)
            ]
df_usdc_filtered

In [ ]:
df_bal_filtered = df_balance_tracker.loc[
            (df_balance_tracker["BLOCK_NUMBER"] > 16027007 - 50) &
            (df_balance_tracker["BLOCK_NUMBER"] < 16027236 + 50)
            ]
df_bal_filtered

In [ ]:
d_start = 92000000
c_start = 63596139.36

data = {
    'id': [],
    "block_number": [],
    "block_timestamp": [],
    "total_debt": [],
    "debt_covered" : [],
    "debt_covered_share": [],
    "total_collateral": [],
    "collateral_covered": [],
    "collateral_covered_share": [],
    "debt_vs_collateral_share": [],
    "liquidator": [],
}
i = 0
current_debt = d_start
current_collateral = c_start
while i < len(liq_filtered):
    debt_covered = float(liq_filtered.iloc[i]['DEBT_TO_COVER_AMOUNT'])
    collateral_covered = float(liq_filtered.iloc[i]['LIQUIDATED_AMOUNT'])

    block_number = int(liq_filtered.iloc[i]['BLOCK_NUMBER'])
    block_timestamp = liq_filtered.iloc[i]['BLOCK_TIMESTAMP']
    liquidator = liq_filtered.iloc[i]['LIQUIDATOR']

    debt_covered_share = debt_covered / current_debt
    collateral_covered_share = collateral_covered / current_collateral
    current_debt -= debt_covered
    current_collateral -= collateral_covered
    data['id'].append(i)
    data['block_number'].append(block_number)
    data['block_timestamp'].append(block_timestamp)
    data['total_debt'].append(current_debt)
    data['debt_covered'].append(debt_covered)
    data['collateral_covered'].append(collateral_covered)
    data['debt_covered_share'].append(debt_covered_share)
    data['collateral_covered_share'].append(collateral_covered_share)
    data['total_collateral'].append(current_collateral)
    data['debt_vs_collateral_share'].append( debt_covered_share - collateral_covered_share )
    data['liquidator'].append( liquidator )



    i += 1

df_processed_liq = pd.DataFrame(data)

df_processed_liq

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig = go.Figure()
fig.update_layout(
    title=f"Liquidation Impact on Position",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)


# fig = fig.add_trace(
#     go.Scatter(
#         x = df_processed_liq.block_timestamp,
#         y = df_processed_liq.collateral_covered_share, 
#         name = "Percent Collateral Sold",
#         mode='lines',
#         marker_color="purple",
#     ),
#     secondary_y=False
# )
fig = fig.add_trace(
    go.Scatter(
        x = df_processed_liq.block_timestamp,
        y = df_processed_liq.debt_covered, 
        name = "Amount of Debt Covered",
        mode='lines',
        marker_color="green",
        line_shape='hv'

    ),
    secondary_y=False
)



fig = fig.add_trace(
    go.Scatter(
        x = df_processed_liq.block_timestamp,
        y = df_processed_liq.debt_vs_collateral_share, 
        name = "Debt % - Collateral %",
        mode='lines+markers',
        marker_color="red",
        line_shape='hv'

    ),
    secondary_y=True
)


fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_initial_liquidation_end'], 
            annotation_text='Liq 1', annotation_position="top left",
            # fillcolor='purple',
            opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_last_liquidation_start'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='Liq 2', annotation_position="top left",
            # fillcolor='purple', 
            opacity=0.25, line_width=0)
fig.update_yaxes(range=[-0.005,0.005 ], secondary_y=True)  

fig.add_hline(y=0, line_width=3, line_dash="dash", line_color="grey", secondary_y=True)

fig.show()
# fig.update_xaxes(range=[0, 9])


In [ ]:
df_processed_liq.sort_values(by=['debt_vs_collateral_share']).tail(60)

In [48]:
def write_dataframe_csv(filename, df):
    print(cwd+"/app/data/storage/"+ filename+'.csv')
    try:
        full_filename = cwd+"/app/data/storage/"+ filename+'.csv'
        df.to_csv(full_filename) 
        return True
    except:
        return False

In [49]:
cwd_temp = os.getcwd()
temp_split = cwd_temp.split('/')
cwd = ""
for x in temp_split:
    if x == 'experiments':
        break
    elif x == '':
        pass
    else:
        cwd += "/"+x

In [38]:
import os

In [ ]:
write_dataframe_csv('liquidation_data', df_processed_liq)